# Поиск потенциально популярного продукта в сегменте компьютерных игр для запуска рекламной кампании в интернет-магазине "Стримчик"   

Интернет-магазине «Стримчик» продаёт по всему миру компьютерные игры. Руководство магазина планирует рекламную кампанию на предстоящий год, в рамках имеющейся стратегии развития бизнеса.   

**Цель исследования:** выявить показатели и закономерности, определяющие успешность игр, для планирования рекламной кампании.   

**Ход исследования:** 
- Найти открытые источники с историческими данными о продажах игр.
- Выполнить предобработку данных (замените названия столбцов при необходимости, преобразовать данные в нужные типы, обработать пропуски при необходимости)
- Провести исследовательский анализ данных (оценить разброс значений по показателям, выявить закономерности, корреляции между показателями. Применительно к конкретной цели оценить продажи игр в зависимости от платформы, жанра, рейтинга покупателей и др.)
- Выдвинуть гипотезы о большей успешности игры (больших продажах) в зависимости от выявленных факторов, проверить их статистически.
- Предоставить аргументированные выводы, полученные в ходе исследования (указать маркетолагам на показатели, которые определяют потенциальную успешность игр).  


## Открываем данные и изучаем общую информацию 

In [ ]:
# Импортируем необходимые для работы библиотеки
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import clean_columns
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Загружаем файл и просмотрим общую информацию о датасете
try:
    df = pd.read_csv('/Users/amirk/Downloads/games.csv')
except:
    df = pd.read_csv('/datasets/games.csv')
    
df.head()

In [ ]:
df.info()

У нас достаточно большой датафрейм, что позволит нам сделать статистически значимые выводы. Большинство столбцов заполнено без пропусков, однако показатели связанные с рейтингами (`Critic_Score`,`User_Score`, `Rating`) имеют пропуски, что вероятно связано с годом выхода игр (на заре продаж компьютерных игр такие параметры не оценивались. Позже проверим данное предположение.)

## Предобработка данных   
### Замена названий столбцов   
Из полученных выше данных мы видим, что названия столбцов написаны не в общепринятом **`змеином_регистре`**. Исправим это!

In [ ]:
df = clean_columns(df) 
df.head()

Замены выполнена корректно!   

### Преобразование типов данных   

При первичном знакомстве с данными, было выявлено несоответсвие типов данных в некоторых столбцах:
- **year_of_release** - нужно заменить на тип `datetime`
- **user_score** - нужно заменить на тип `float64`   

Выполним преобразование!

In [ ]:
df['year_of_release'] = pd.to_datetime(df['year_of_release'], format='%Y')

# преобразование типа данных в столбце user_csore приводит к ошибке
# посмотрим какие значения встречаются в данном столбце
df['user_score'].value_counts()

In [ ]:
# Получено 2424 строковых значения 'tbd' - to be decided/determined
# эти значения эще не посчитаны, возвможно это данные за последний год. Проверим.
tbd_stat = df[df['user_score'] == 'tbd']['year_of_release'].value_counts().sort_index()
print(tbd_stat)

Видно, что значение **tbd** встречается в разные годы. с 2012 по 2016 гг таких значений мало, статистика ведется более полноценно. За предыдущие годы данная статистика велась достаточно плохо.
Возможно продажи данных игр достаточно низкие, их процент от общего количества продаж очень мал и мы сможем удалить данные значения из датасета. Проверим.

In [ ]:
tbd_stat = tbd_stat.reset_index().rename(columns={'index': 'year_of_release','year_of_release': 'tbd_per_year'})

#создадис список стран для использования
country_list = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
for j in country_list:
    tbd_list = []
    for i in tbd_stat['year_of_release']:
        a = (df.loc[(df['user_score'] == 'tbd') & (df['year_of_release'] == i)][j].sum()) / (df[df['year_of_release'] == i][j].sum())
        tbd_list.append((a * 100).round(2))
    tbd_stat[j] = tbd_list
tbd_stat

Видно, что за все года вклад игр со значением **tbd** в столбце `user_score` в общий объем продаж во всех регионах составляет от 2% до 15% в зависимости от региона продаж, поэтому данные позиции можно заменить на `NaN`, они не вносят большой вклад в общий объем продаж, особенно если предположить, что нам для рассчета понадобятся данные за актуальный период с 2012 по 2016гг. Далее заменим все цифровые значения на тип `float`.

In [ ]:
df.loc[df['user_score'] == 'tbd', 'user_score'] = np.NaN
df['user_score'] = df['user_score'].astype('float')
df.info()

### Обработка пропусков   

Вспомним, где были пропуски в датафрейме

In [ ]:
df.isna().sum()

Проверим более подробнее пропуски в каждом столбце, по возможности удалим.

In [ ]:
df[df['name'].isna() == True]

Обе строчки с пропусками являются абсолютно неинформативными, нет названия игры, жанра и оценок. Смело удаляем данные строки.

In [ ]:
df = df.dropna(subset=['name'])
df.info()
df.isna().sum()

Удалив пропуски в именах, мы удалили и пропуски в жанре

In [ ]:
# Оценим пропуски в year_of_release
df[df['year_of_release'].isna() == True]

Видно, что у многих игр в названии стоит год выпуска, можем заполнить пропуски этими данными.

In [ ]:
drop_years_list = df[df['year_of_release'].isna() == True]['name'].unique()
for i in drop_years_list:
    if (str(i).split(sep=' ')[-1].isnumeric() == True) and (len(str(i).split(sep=' ')[-1]) == 4):
        df.loc[df['name'] == i, 'year_of_release'] = (str(i).split(sep=' ')[-1])
        
        df.loc[df['name'] == i, 'year_of_release'] = pd.to_datetime(df.loc[df['name'] == i, 'year_of_release'], format='%Y')
df.isna().sum()

Не сильно, но уменьшили количество пропусков. 

In [ ]:
# Оценим пропуски в critic_score 
# Особо интересует распределение пропусков по годам

print(df[df['critic_score'].isna() == True]['year_of_release'].value_counts().sort_index())

# Заодно оценим распределение показателя
df['critic_score'].plot(kind='hist', bins=100, title='Распределение значений critic_score', grid=True)
plt.show()


Видим, что пропусков достаточно в каждом из годов. Заполнять пропуски медианным или средним значением в зависимости от года выхода, жанра, платформы, названия игры и тд. - нельзя! Потому что выход новой игры или новой версии абсолютно не означает схожие оценки с предыдущей версией. Новая версия игры может быть оценена намного выше или ниже медианного значения.   
Просто удалить пропуски мы не можем, потому что это около половины датасета, мы исказим результаты. Поэтому оставим пропуски!   
Также в данных нет выбросов и атипичных значений.

In [ ]:
# Так же оценим пропуски в user_score
print(df[df['user_score'].isna() == True]['year_of_release'].value_counts().sort_index())
df['user_score'].plot(kind='hist', bins=10, title='Распределение значений user_score', grid=True);
plt.show()

Аналогичная ситуация как и в `critic_score`. Пропуски оставляем, атипичных значений в данных нет!

In [ ]:
# Ожидаем, что в rating будет аналогичная ситуация

print(df[df['rating'].isna() == True]['year_of_release'].value_counts().sort_index())
df.pivot_table(index='rating', values='name', aggfunc='count').sort_values(by='name', ascending=False).plot(kind='bar', title='Распределение значений rating', grid=True, legend=False);
plt.show()

Ожидания оправдались, аналогичная закономерность. Пропуски оставляем, атипичных значений в данных нет!

Пропуски в year_of_release могли возникнуть из-за ошибки в системе сбора информации, возможно при переносе данных их одной базы данных в общую.   
Пропуски в столбцах с оценками и рейтингом могли также возникнуть по причине ошибок в системе сбора информации, некоторые малопулярные игры могли быть просто не оценены пользователями и по рейтингу или авторы игр просто не отправляли разработанную игру на получение оценки и рейтинга.

### Посчет суммарных продаж во всех регионах   

Создадим столбец `total_sales` и внесем в него сумму продаж по всем регионам.

In [ ]:
df['total_sales'] = df[country_list].sum(axis=1) 
df.head()

**Промежуточные итоги**   
- заменили названия столбцов на общепринятый формат
- заменили типы данных
- заполнили пропуски там, где это возможно. В столбце user_score заменили значения **tbd** на **NaN**, чтобы можно было вести работу с цифрами и не потерять около половины данных. Данное значение **tbd** вероятно возникло из-за ошибок системы сбора статистики. В столбцах с оценками пропуски оставили, так-так оценки игры среди пользователей предугадать сложно, в разных версиях игр могут быть 'взлеты' и 'падения'.
- посчитали общий обьем продаж по каждой позиции.

## Исследовательский анализ данных

Нам нужно выделить **актуальный период** в годах, данные за который мы могли бы использовать для прогнозирования потенциально успешного продукта в 2017гг. Это все связано с изменением тенденций в мире компьютерных игр.

### Оценим, сколько игр выспускалось в разные годы

In [ ]:
df.groupby('year_of_release')['name'].count().plot(kind='barh', figsize=(15,10))
plt.xlabel('Количество игр')
plt.title('Распределение количества выпущенных игр по годам')
plt.show()


Видно, что по мере развития индустрии компьютерных игр из года в год увеличивалось количество выпускаемых игр, при этом пик пришелся на 2009-2010гг.С 2005 по 2011 гг в год выпускалось около 1000-1400 игр. Однако более современный этап - 2012-2016гг, показывает, что количесвто выпускаемых компьютерных игр сократилось практически вдвое до 500-600 игр в год.   
Это может быть связано c:
- ростом популярности мобильных устройств и игр на них
- переключением производителей с показателей количества, на показатели качества. Стало важным сделать качественную игру с хорошей графикой и сюжетом.
- последствиями экономического кризиса.   

Все это нам говорит о том, что в качестве **актуального периода** следует рассмаривать период с 2012 по 2016гг.

### Оценим продажи игр в зависимости от платформы

In [ ]:
f, ax = plt.subplots(figsize=(10, 7))
platform_pt = df.pivot_table(index=df['year_of_release'].dt.year, columns='platform', values='total_sales', aggfunc='sum')
sns.heatmap(platform_pt)
plt.xlabel('Игровые платформы')
plt.ylabel('Год выпуска')
plt.title('Погодовое распределение объема продаж игр (млн) в зависимости от платформы')
plt.show()
# Также посмотрим наиболее популярные платформы за все время
display(df.pivot_table(index='platform', values='total_sales', aggfunc='sum').sort_values(by='total_sales', ascending=False).head(10))

# Дополнительно оценим популярность платформ за последние 5 лет
display(df[df['year_of_release'] >= pd.Timestamp(ts_input='2012-01-01')].pivot_table(index='platform', values='total_sales', aggfunc='sum').sort_values(by='total_sales', ascending=False))

Представленная тепловая карта хорошо показывает эволюцию игровых платформ и наиболее популярные современные платформы. Видно, что жизненный цикл определенной платформы в районе 10-12 лет, пик популярности приходится на 3-5 годы после выпуска платформы, новая версия платформы выходит приблизительно через 6-7 лет. Среди функционирующих игровых платформ наиболее популярными в последние 5 лет являются:
- PS4 (явный лидер по объемам продаж, отмечается рост из года в год (2016г не полный))
- PS3 (однако отмечается спад продаж в последние годы)	
- X360 (однако отмечается спад продаж в последние годы)	
- 3DS (однако отмечается спад продаж в последние годы)	
- XOne (набирает популярность)
- WiiU (отмечается спад продаж)
- PC (стабильные продажи из года в год)	
- PSV (стабильно низкие объемы продаж)	
- Wii (жизненный цикл платформы близится к концу)   

Таким образом мы пришли к выводам, что **актуальный период** для наших дальнейших расчетов является **период с 2012 по 2016гг**, а к наиболее перспективным игровым платформам относятся **PS4** и **XOne**.

### Актуализируем данные
Сохраним в датафрейм `df_actual` данные за последние 5 лет.

In [ ]:
df_actual = df[df['year_of_release'] >= pd.Timestamp(ts_input='2012-01-01')]
df_actual.head()

### Построим график «ящик с усами» по глобальным продажам игр в разбивке по платформам.

In [ ]:
sns.boxplot(data=df_actual, x='platform', y="total_sales")
plt.xlabel('Игровые платформы')
plt.ylabel('Объем продаж (млн)')
plt.title('Распределение продаж игр в зависимости от платформы')
plt.show()

Видим, что самая популярная игра за последние 5 лет набрала более 20 млн продаж. В среднем игровые хиты первой пятерки среди игровых платформ набирали 10-15 млн продаж, для замыкающей пятерки - в районе 5-7 млн продаж.    
Ограничим выбросы для более подробной оценки медиан и квартилей.

In [ ]:
sns.boxplot(data=df_actual, x='platform', y="total_sales")
plt.xlabel('Игровые платформы')
plt.ylabel('Объем продаж (млн)')
plt.ylim(0, 2,5)
plt.title('Распределение продаж игр в зависимости от платформы')
plt.show()

Самая высокая медиана продаж у платформы **X360**, расположение квартилей так же показывает, что на данной платформе игры в среднем набирают больше продаж, чем на других платформах. В целом у популярных платформ медиана продаж для отдельной игры находится в районе 220-230 тыс. копий, верхний квартиль в районе 600-700 тыс.

### Оценим взаимосвязь оценок критиков и пользователей с объемами продаж на разных платформах

In [ ]:
# Для начала оценим самую перспективную платформу PS4
df_actual[df_actual['platform'] == 'PS4'].plot(x='total_sales', 
                                               y='critic_score', 
                                               kind='scatter',
                                               title=('Взаимосвязь продаж игр с оценкой критиков на игровой платформе PS4'),
                                               alpha=0.15, 
                                               xlim=(0, 3))
plt.show()
df_actual[df_actual['platform'] == 'PS4'].plot(x='total_sales', 
                                               y='user_score', 
                                               kind='scatter',
                                               title=('Взаимосвязь продаж игр с оценкой пользователей на игровой платформе PS4'),
                                               alpha=0.15, 
                                               xlim=(0, 20))
plt.show()
print('''Коэффициент корреляции Пирсона между оценкой критиков и объемом продаж игр
на платформe PS4 равен:''', df_actual[df_actual['platform'] == 'PS4']['total_sales'].corr(df_actual[df_actual['platform'] == 'PS4']['critic_score']).round(2))
print('''Коэффициент корреляции Пирсона между оценкой пользователей и объемом продаж игр
на платформt PS4 равен:''', df_actual[df_actual['platform'] == 'PS4']['total_sales'].corr(df_actual[df_actual['platform'] == 'PS4']['user_score']).round(2))

Мы видим слабую взаимосвязь межу оценками критиков, пользователей и показателем объема продаж игр на данной платформе. При этом можно отметить, что чем больше продаж у игр, тем меньше встречается игр с низкой оценкой критиков, вероятно здесь нелинейная корреляция.  
Аналогично оценим оставшиеся платформы.

In [ ]:
for i in df_actual.loc[df_actual['platform'] != 'PS4', 'platform'].unique():
    df_actual[df_actual['platform'] == i].plot(x='total_sales', 
                                                   y='critic_score', 
                                                   kind='scatter',
                                                   title=('Взаимосвязь продаж игр с оценкой критиков на игровой платформе ' + i),
                                                   alpha=0.15, 
                                                   xlim=(0, 3))
    plt.show()
    df_actual[df_actual['platform'] == i].plot(x='total_sales', 
                                                   y='user_score', 
                                                   kind='scatter',
                                                   title=('Взаимосвязь продаж игр с оценкой пользователей на игровой платформе ' + i),
                                                   alpha=0.15, 
                                                   xlim=(0, 20))
    plt.show()
    print(f'''Коэффициент корреляции Пирсона между оценкой критиков и объемом продаж игр
на платформe {i} равен:''', df_actual[df_actual['platform'] == i]['total_sales'].corr(df_actual[df_actual['platform'] == i]['critic_score']).round(2))
    print(f'''Коэффициент корреляции Пирсона между оценкой пользователей и объемом продаж игр
на платформt {i} равен:''', df_actual[df_actual['platform'] == i]['total_sales'].corr(df_actual[df_actual['platform'] == i]['user_score']).round(2))

Для остальных платформ выявлены схожие результаты. Четкой линейной взаимосвязи между объемом продаж и оценками критиков и пользователей нет!

### Оценим распреденение объема продаж игр в зависимости от жанра

In [ ]:
f, ax = plt.subplots(figsize=(10, 3))
sns.heatmap(df_actual.pivot_table(index=df['year_of_release'].dt.year, columns='genre', values='total_sales', aggfunc='median'))
plt.xlabel('Игровые жанры')
plt.ylabel('Год выпуска')
plt.title('Распределение объема продаж игр (млн) в зависимости от жанра')
plt.show()
print(df_actual.pivot_table(index='genre', values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False))

За выбранный актуальный период наибольшая средняя прибыль 1 игры у жанров:   
 - `Shooter`   
 - `Sports`   
 - `Role-Playing`   
 - `Racing`   
 - `Fighting`   
 
А у абсолютного лидера по общему объему продаж - жанра `Action`, показатель средней прибыли за 1 игру находится лишь на 6 месте.

## Портрет среднестатистического пользователя в зависимости от региона продаж

###   Оценим ТОП-5 игровых платформ среди пользователей различных регионов

In [ ]:
#создадим словарь с регионами продаж
region_dict = {'na':'Северная Америка', 'eu':'Европа', 'jp':'Япония'}

#Создадим функциюдля построения свойдной таблицы топ-5 чего-либо по регионам
def top5_table(smth):
    top5_smth_by_region = pd.DataFrame()
    for region in region_dict:
        top5_smth_by_region = top5_smth_by_region.merge(df_actual.pivot_table(index=smth, 
                                                                                      values=region + '_sales', 
                                                                                      aggfunc='sum')
                                                                .sort_values(by=region + '_sales', 
                                                                             ascending=False)
                                                                .head()
                                                                .reset_index()
                                                                .rename(columns={smth:region_dict[region]}), 
                                                                        left_index=True, right_index=True, how='right')
    return top5_smth_by_region

In [ ]:
top5_table('platform')

In [ ]:
# Подкрепим таблицу диаграммами
# Напишем функцию для выводы диаграмм
def top5_diagramm(smth, title):
    for region in region_dict:
        plt.figure(0)
        (df_actual.pivot_table(index=smth, values=region + '_sales', aggfunc='sum')
         .sort_values(by=region + '_sales', ascending=False).head()
         .plot(subplots=True, 
               kind='pie', 
               figsize=(6, 6), 
               ylabel='', 
               title=title + region_dict[region]));
    return

In [ ]:
top5_diagramm('platform', 'ТОП-5 игровых платформ по продажам - ')

Видно, что предпочтения в выборе игровой платформы у пользователей различных регионов значительно различаются. Пользователи из Северной Америки предпочитают **X360**, из Европы - **РS4**, из Японии - **3DS**. При этом ТОП-5 игровых платформ в Америке и Европе одинаковы, а в Японии вместо **X360** и **XOne** (разработанных в США) присутсвуют только местные платформы, подавляющее большинство приходится на **3DS**.

### Оценим ТОП-5 игровых жанров среди пользователей различных регионов

In [ ]:
top5_table('genre')

In [ ]:
top5_diagramm('genre', 'ТОП-5 игровых жанров по продажам - ')

С игровыми жанрами также отмечается схожие предпочтения вреди пользователей Северной Америки и Европы. В обоих регионах превалирующее большинство пользователей предпочитают жанры **Action** и **Shooter**. В Японии жанр **Action** также пользуется большой популярностью, однако первое место занимает жанр **Role-Playing**.

### Оценим продажи игр по регионам в зависимости от рейтинга

In [ ]:
# Заменим пропуски в rating на значение-заглушку 'unknown'
df_actual['rating'].fillna(value='unknown', inplace=True)

top5_table('rating')

In [ ]:
# Подкрепим таблицу диаграммами
top5_diagramm('rating', 'ТОП-5 возрастных рейтингов по продажам - ')

Около 1/3 объема продаж в Северной Америке и Европе приходится на игры с рейтингом **М** (для лиц старше 17 лет), что говорит о пристрастии данных пользователей к более агрессивным играм. Около четверти приходится на игры с рейтингом **Е** (для всех), чуть меньше занимают игры, не имеющие рейтинга ESRB и оставшуюся часть примерно поровно делят игры с рейтингом **T** (для лиц старше 17 лет) и **Е10+** (для лиц старше 10 лет). В Японии более половины игр не имеют рейтинга ESRB, примерно 1/4 объема продаж приходится на игры с рейтингом **Т** и **Е**, оставшаяся часть приходится на рейтинги **М** и **Е10+**.    

Данные результаты показывают, что производителю игр для рынка Северной Америки и Европы важно получить рейтинг ESRB, так как среди пользователей пользуется большей популярность игры с установленным рейтингом. Обратная ситуация в Японии, где наличие рейтинга ESRB особо не влияет на выбор покупателя (большая часть проданных игр не имеет рейтинга ESRB), поэтому производители могут не получать рейтинг для своей продукции.

## Проверка гипотез

### Средние пользовательские рейтинги платформ Xbox One и PC одинаковые?
Предположим, что средние пользовательские рейтинги платформ **Xbox One** и **PC** равны.   
Нулевая гипотеза: средняя оценка пользователей **Xbox One** = средняя оценка пользователей **PC**   
Альтернативная гипотеза: средняя оценка пользователей **Xbox One** ≠ средняя оценка пользователей **PC**   
Уровень значимости = 0,05

В качестве статистического метода в данном случае уместно использовать t-test, так как наши данные количественные, величина выборки более 30 наблюдений (распределение Стьюдента близится к нормальному), генеральные сосокупности независимые. Равенство дисперсий нам неизвестно, поэтому отразим это при проведении t-теста (equal_var=False).

In [ ]:
alpha = .05

result = st.ttest_ind((df_actual.loc[df_actual['platform'] == 'XOne']['user_score'].dropna()), 
                      (df_actual.loc[df_actual['platform'] == 'PC']['user_score'].dropna()), 
                      equal_var=False)
if result[1] < alpha:
    print(f'Отвергаем нулевую гипотезу, значение p-value = {result[1]:.3f}')
else:
    print(f'Нулевую гипотезу отвергнуть не удалось, значение p-value = {result[1]:.3f}')

    # Дополнительно оценим описательные статистики для указанных данных
print('Среднее, медиана и квартили для Xbox One')
print(df_actual.loc[df_actual['platform'] == 'XOne']['user_score'].describe())
print('Среднее, медиана и квартили для PC')
print(df_actual.loc[df_actual['platform'] == 'PC']['user_score'].describe())

Мы проверили гипотезу при помощи двустороннего t-теста. По итогам проверки **на основе имеющихся данных** мы пришли к выводу, что средние оценки пользователей платформ **Xbox One** и **PC** **равны**, значение p-value = 0,55. Об этом также могут свидетельствовать схожие значения средних и медиан в данных группах.

### Средние пользовательские рейтинги жанров Action и Sports разные?

Предположим, что средние пользовательские рейтинги жанров **Action** и **Sports** разные.
Нулевая гипотеза: средняя оценка пользователей жанра **Action** = средняя оценка пользователей жанра **Sports**
Альтернативная гипотеза: средняя оценка пользователей жанра **Action** ≠ средняя оценка пользователей жанра **Sports**  
Уровень значимости = 0,05

In [ ]:
alpha = .05

result = st.ttest_ind((df_actual.loc[df_actual['genre'] == 'Action']['user_score'].dropna()), 
                      (df_actual.loc[df_actual['genre'] == 'Sports']['user_score'].dropna()), 
                      equal_var=False)
if result[1] < alpha:
    print(f'Отвергаем нулевую гипотезу, значение p-value = {result[1]:.3f}')
else:
    print(f'Нулевую гипотезу отвергнуть не удалось, значение p-value = {result[1]:.3f}')

# Дополнительно оценим описательные статистики для указанных данных
print('Среднее, медиана и квартили для жанра Action')
print(df_actual.loc[df_actual['genre'] == 'Action']['user_score'].describe())
print('Среднее, медиана и квартили для жанра Sports')
print(df_actual.loc[df_actual['genre'] == 'Sports']['user_score'].describe())

Полученные результаты позволяют нам отвергнуть нулевую гипотезу. Следовательно наше предположение о том, что средние пользовательские рейтинги жанров **Action** и **Sports** разные, оказалось верным! Значение p-value менее 0,001 и стремится к нулю.

# Итоги проведенного исследования

В ходе исследования нам было необходимо выявить показатели и закономерности, определяющие успешность игр, для планирования рекламной кампании в интернет-магазине 'Стримчик'.   

В ходе исследования мы использовали открытые данные по продажам игр за разные года вплоть до 2016г. 
Нам пришлось выполнить предобработку данных так как данные были неподготовлены для анализа:
- заменили названия столбцов на общепринятый **`змеиный_регистр`**;
- преобразовали данные в нужные типы (**year_of_release** в тип `datetime`, **user_score** в тип `float64`, обработать пропуски при необходимости);
- заполнили пропуски (В столбце **user_score** заменили значения **tbd** на **NaN**, чтобы можно было вести работу с цифрами и не потерять около половины данных. Данное значение **tbd** вероятно возникло из-за ошибок системы сбора статистики. В столбцах с оценками пропуски оставили, так-так оценки игры среди пользователей предугадать сложно, в разных версиях игр могут быть 'взлеты' и 'падения'.);
- посчитали общий объем продаж по каждой позиции.  

Далее перед нами стояла задача провести исследовательский анализ данных, в ходе которого было сделано:
- проанализировано, сколько игр выпускалось в разные годы (по мере развития индустрии компьютерных игр из года в год увеличивалось количество выпускаемых игр, при этом пик пришелся на 2009-2010гг.С 2005 по 2011 гг в год выпускалось около 1000-1400 игр. Однако более современный этап - 2012-2016гг, показывает, что количесвто выпускаемых компьютерных игр сократилось практически вдвое до 500-600 игр в год.); 
- проанализировано, как менялись продажи по платформам (построенная тепловая карта хорошо показала эволюцию игровых платформ и наиболее популярные современные платформы. Видно, что жизненный цикл определенной платформы в районе 10-12 лет, пик популярности приходится на 3-5 год после выпуска платформы, новая версия платформы выходит приблизительно через 6-7 лет. Среди функционирующих игровых платформ наиболее популярными в последние 5 лет являются:
   - PS4 (явный лидер по объемам продаж, отмечается рост из года в год (2016г не полный))
   - PS3 (однако отмечается спад продаж в последние годы)	
   - X360 (однако отмечается спад продаж в последние годы)	
   - 3DS (однако отмечается спад продаж в последние годы)	
   - XOne (набирает популярность)
   - WiiU (отмечается спад продаж)
   - PC (стабильные продажи из года в год)	
   - PSV (стабильно низкие объемы продаж)	
   - Wii (жизненный цикл платформы близится к концу)
- сделаны выводы, что **актуальный период** для наших дальнейших расчетов является **период с 2012 по 2016гг**, а к наиболее перспективным игровым платформам относятся **PS4** и **XOne**; 
- построен график «ящик с усами» по глобальным продажам игр в разбивке по платформам (самая популярная игра за последние 5 лет набрала более 20 млн продаж. В среднем игровые хиты первой пятерки среди игровых платформ набирали 10-15 млн продаж, для замыкающей пятерки - в районе 5-7 млн продаж. Самая высокая медиана продаж у платформы **X360**, расположение квартилей так же показывает, что на данной платформе игры в среднем набирают больше продаж, чем на других платформах. В целом у популярных платформ медиана продаж для отдельной игры находится в районе 220-230 тыс. копий, верхний квартиль в районе 600-700 тыс.);
- проанализировано влияние отзывов пользователей и критиков на продажи внутри разных игровых платформ, построены диаграммы рассеяния и посчитана корреляция между отзывами и продажами (Для самой популярной платформы **PS4** выявлена слабая взаимосвязь межу оценками критиков, пользователей и показателем объема продаж игр. При этом можно отметить, что чем больше продаж у игр, тем меньше встречается игр с низкой оценкой критиков, вероятно здесь нелинейная корреляция. Для остальных платформ выявлены схожие результаты. Четкой линейной взаимосвязи между объемом продаж и оценками критиков и пользователей нет!);
- проанализировано общее распределение игр по жанрам (Мы видим, что лидерами продаж за последние 5 лет являются игры жанра `Action`, `Shooter`, `Role-Playing`, `Sports`. При этом нужно отметить, что у игр жанра `Action`в последние годы популярность уменьшается, а популярность игр `Shooter`, `Role-Playing`, `Sports` остается примерно одинаковой. Наименее популярными играми с наименьшим объемом продаж являются игры жанра `Puzzle`, `Strategy`, `Adventure`.).   

Далее для возможности запуска таргетированной рекламы мы оценили портрет средненго пользователя каждого региона. Определили для пользователей каждого региона (NA (Северная Америка), EU (Европа), JP (Япония)):
- ТОП-5 популярных игровых платформ (Видно, что предпочтения в выборе игровой платформы у пользователей различных регионов значительно различаются. Пользователи из Северной Америки предпочитают **X360**, из Европы - **РS4**, из Японии - **3DS**. При этом ТОП-5 игровых платформ в Америке и Европе одинаковы, а в Японии вместо **X360** и **XOne** (разработанных в США) присутсвуют только местные платформы, подавляющее большинство приходится на **3DS**.);
- ТОП-5 популярных жанров (С игровыми жанрами также отмечается схожие предпочтения вреди пользователей Северной Америки и Европы. В обоих регионах превалирующее большинство пользователей предпочитают жанры **Action** и **Shooter**. В Японии жанр **Action** также пользуется большой популярностью, однако первое место занимает жанр **Role-Playing**.);
- влияние рейтинга ESRB на продажи в отдельном регионе (Около 1/3 объема продаж в Северной Америке и Европе приходится на игры с рейтингом **М** (для лиц старше 17 лет), что говорит о пристрастии данных пользователей к более агрессивным играм. Около четверти приходится на игры с рейтингом **Е** (для всех), чуть меньше занимают игры, не имеющие рейтинга ESRB и оставшуюся часть примерно поровно делят игры с рейтингом **T** (для лиц старше 17 лет) и **Е10+** (для лиц старше 10 лет). В Японии более половины игр не имеют рейтинга ESRB, примерно 1/4 объема продаж приходится на игры с рейтингом **Т** и **Е**, оставшаяся часть приходится на рейтинги **М** и **Е10+**. Данные результаты показывают, что производителю игр для рынка Северной Америки и Европы важно получить рейтинг ESRB, так как среди пользователей пользуется большей популярность игры с установленным рейтингом. Обратная ситуация в Японии, где наличие рейтинга ESRB особо не влияет на выбор покупателя (большая часть проданных игр не имеет рейтинга ESRB), поэтому производители могут не получать рейтинг для своей продукции.). 

На следующем этапе проверили выдвинутые нами гипотезы с применением методов сатитстической обработки данных:
- Гипотеза 1. Средние пользовательские рейтинги платформ **Xbox One** и **PC** равны.   
  Нулевая гипотеза: средняя оценка пользователей **Xbox One** = средняя оценка пользователей **PC**   
  Альтернативная гипотеза: средняя оценка пользователей **Xbox One** ≠ средняя оценка пользователей **PC**   
  Уровень значимости = 0,05
  Мы проверили гипотезу при помощи двустороннего t-теста. По итогам проверки **на основе имеющихся данных** мы пришли к выводу, что средние оценки пользователей платформ **Xbox One** и **PC** **равны**, значение p-value = 0,55. Об этом также могут свидетельствовать схожие значения средних и медиан в данных группах.
- Гипотеза 2. Средние пользовательские рейтинги жанров `Action` и `Sports` разные.   
  Нулевая гипотеза: средняя оценка пользователей жанра `Action` = средняя оценка пользователей жанра `Sports`   
  Альтернативная гипотеза: средняя оценка пользователей жанра `Action` ≠ средняя оценка пользователей жанра `Sports`    
  Уровень значимости = 0,05   
  Полученные результаты позволяют нам отвергнуть нулевую гипотезу. Следовательно наше предположение о том, что средние пользовательские рейтинги жанров `Action` и `Sports` разные, оказалось верным! Значение p-value менее 0,001 и стремится к нулю.   

**Заключительные выводы:**
- При планировании рекламной кампании необходимо делать акцент на наиболее перспективные игровые платформы **PS4** и **XOne**, среди игровых жанров предпочтение необходимо отдать `Action`, `Shooter`, `Role-Playing`, `Sports`.
- При возвожности запуска таргетированной рекламы необходимо учитывать особенности региона продаж:
    + Северная Америка (NA) - необходимо делать акцент на игровые платформы **X360**, **PS4**, **PS3**, **XOne**, **3DS**, игровые жанры `Action`, `Shooter`, `Sports`, рейтинг ESRB **M** и **E**.
    + Европа (EU) - необходимо делать акцент на игровые платформы **PS4**, **PS3**, **X360**, **XOne**, **3DS**, игровые жанры `Action`, `Shooter`, `Sports`, рейтинг ESRB **M** и **E**.
    + Япония (JP) - необходимо делать акцент на игровые платформы  **3DS**, **PS3**, **PSV**, **PS4**, **WiiU**, игровые жанры `Role-Playing`, `Action`, `Misc`, рейтинг ESRB не особо важен, а среди игр с рейтингом превалируют **T** и **E**.
    
    
    
    

 